In [28]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.layers import Input, Conv2D, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.regularizers import l2

In [29]:
train_path='H:/20_class_8k/Train'
test_path="H:/20_class_8k/test"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [30]:
print("The Training Class --->",class_names)
print("\n\nThe Test Classes----> ",class_names_test)

The Training Class ---> ['ইউটার্ন', 'ইউটার্ন নেওয়া নিষেধ', 'একদিকে চলাচল', 'ওভারটেকিং নিষেধ', 'ট্র্যাক প্রবেশ নিষেধ', 'ডানদিকে মোড় নেওয়া নিষেধ', 'থামানো নিষেধ', 'পার্কিং', 'পার্কিং নিষেধ', 'প্রবেশ নিষেধ', 'বামদিকে মোড় নেওয়া নিষেধ', 'বাস স্ট্যান্ড', 'মসজিদ', 'রিকশা চলাচল নিষেধ', 'সামনে ডানদিকে আচমকা মোড় আছে', 'সামনে পথচারী পারাপার', 'সামনে বামদিকে পার্শ্ব রাস্তা আছে', 'সামনে স্কুল', 'হর্ন বাজানো নিষেধ', 'হাসপাতাল']


The Test Classes---->  ['ইউটার্ন', 'ইউটার্ন নেওয়া নিষেধ', 'একদিকে চলাচল', 'ওভারটেকিং নিষেধ', 'ট্র্যাক প্রবেশ নিষেধ', 'ডানদিকে মোড় নেওয়া নিষেধ', 'থামানো নিষেধ', 'পার্কিং', 'পার্কিং নিষেধ', 'প্রবেশ নিষেধ', 'বামদিকে মোড় নেওয়া নিষেধ', 'বাস স্ট্যান্ড', 'মসজিদ', 'রিকশা চলাচল নিষেধ', 'সামনে ডানদিকে আচমকা মোড় আছে', 'সামনে পথচারী পারাপার', 'সামনে বামদিকে পার্শ্ব রাস্তা আছে', 'সামনে স্কুল', 'হর্ন বাজানো নিষেধ', 'হাসপাতাল']


In [31]:
data_dir = "H:/Resnet50/test"

files = os.listdir(data_dir)
dataset_path = [os.path.basename(file) for file in files]
print(dataset_path)

['No left turn', 'No Overtaking', 'NO Parking', 'No rickshaws', 'No right turn', 'No trucks', 'No U turn', 'No use of horn', 'One way street', 'Parking', 'Pedestrian Crossing', 'School', 'Sharp bend to the right ahead', 'Side road on the left ahead', 'U-turn']


In [32]:
# import os
# import cv2
# import random
# import matplotlib.pyplot as plt

# # Define the path to the directory containing the images
# data_dir = "H:/VGG-16/test"

# # Select two random classes from the directory
# class_names = random.sample(os.listdir(data_dir), 2)

# # Display two random images from each class
# for class_name in class_names:
#     class_dir = os.path.join(data_dir, class_name)
#     image_name = random.choice(os.listdir(class_dir))
#     image_path = os.path.join(class_dir, image_name)

#     # Read and display the image using OpenCV and Matplotlib
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    
#     plt.figure()
#     plt.imshow(image)
#     plt.title(f"Class: {class_name}")
#     plt.axis('off')

# plt.show()
import os
import cv2
import random
import matplotlib.pyplot as plt

# Define the path to the directory containing the images
data_dir = "H:/20_class_8k/test"

# Select two random classes from the directory
class_names = random.sample(os.listdir(data_dir), 2)

# Display two random images from each class
for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    image_name = random.choice(os.listdir(class_dir))
    image_path = os.path.join(class_dir, image_name)

    # Read the image using OpenCV
    image = cv2.imread(image_path)

    if image is not None:
        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Display the image using Matplotlib
        plt.figure()
        plt.imshow(image)
        plt.title(f"Class: {class_name}")
        plt.axis('off')
    else:
        print(f"Failed to read image: {image_path}")

plt.show()


Failed to read image: H:/20_class_8k/test\বাস স্ট্যান্ড\bus stop_0_8845.png
Failed to read image: H:/20_class_8k/test\প্রবেশ নিষেধ\noentry2_output_0_5450.png


In [33]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()
val_datagen=ImageDataGenerator()

In [34]:
# train_generator = train_datagen.flow_from_directory("H:/Dataset/train",target_size=(224, 224),batch_size=32,shuffle=True,class_mode='binary')
# test_generator = test_datagen.flow_from_directory("H:/Dataset/test",target_size=(224,224),batch_size=32,shuffle=False,class_mode='binary')


train_generator = train_datagen.flow_from_directory(
    "H:/Resnet50/Train",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True,
    class_mode='categorical',
    #subset='training'
)

test_generator = test_datagen.flow_from_directory(
    "H:/Resnet50/test",
    target_size=(224, 224),
    batch_size=50,
    shuffle=False,
    class_mode='categorical',
    #subset='validation'
)

# Generate batches of validation data
val_generator = val_datagen.flow_from_directory(
    "H:/Resnet50/validation",
    target_size=(224, 224),
    batch_size=50,
    shuffle=False,
    class_mode='categorical'
)


Found 4200 images belonging to 15 classes.
Found 1200 images belonging to 15 classes.
Found 600 images belonging to 15 classes.


In [35]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [20]:
def convolutional_block(X, f, filters, stage, block, s=1):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [21]:
# def ResNet50(input_shape=(224, 224, 3)):

#     X_input = Input(input_shape)

#     X = ZeroPadding2D((3, 3))(X_input)

#     X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
#     X = BatchNormalization(axis=3, name='bn_conv1')(X)
#     X = Activation('relu')(X)
#     X = MaxPooling2D((3, 3), strides=(2, 2))(X)

#     X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
#     X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
#     X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


#     X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
#     X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
#     X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
#     X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

#     X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
#     X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

#     X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
#     X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
#     X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

#     X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
#     model = Model(inputs=X_input, outputs=X, name='ResNet50')

#     return model

In [22]:
def ResNet50(input_shape=(224, 224, 3), num_classes=15):
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.001))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [23]:
base_model = ResNet50(input_shape=(224, 224, 3))

In [24]:
# headModel = base_model.output
# headModel = Flatten()(headModel)
# headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
# headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
# headModel = Dense( 47,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = base_model.output
headModel = Flatten()(headModel)
headModel = Dense(256, activation='relu', name='fc1')(headModel)
headModel = Dense(128, activation='relu', name='fc2')(headModel)
headModel = Dense(15, activation='softmax', name='fc3')(headModel)


In [25]:
model = Model(inputs=base_model.input, outputs=headModel)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

                                                                                                  
 activation_8 (Activation)      (None, 55, 55, 64)   0           ['bn2c_branch2b[0][0]']          
                                                                                                  
 res2c_branch2c (Conv2D)        (None, 55, 55, 256)  16640       ['activation_8[0][0]']           
                                                                                                  
 bn2c_branch2c (BatchNormalizat  (None, 55, 55, 256)  1024       ['res2c_branch2c[0][0]']         
 ion)                                                                                             
                                                                                                  
 add_2 (Add)                    (None, 55, 55, 256)  0           ['bn2c_branch2c[0][0]',          
                                                                  'activation_6[0][0]']           
          

 ion)                                                                                             
                                                                                                  
 add_5 (Add)                    (None, 28, 28, 512)  0           ['bn3c_branch2c[0][0]',          
                                                                  'activation_15[0][0]']          
                                                                                                  
 activation_18 (Activation)     (None, 28, 28, 512)  0           ['add_5[0][0]']                  
                                                                                                  
 res3d_branch2a (Conv2D)        (None, 28, 28, 128)  65664       ['activation_18[0][0]']          
                                                                                                  
 bn3d_branch2a (BatchNormalizat  (None, 28, 28, 128)  512        ['res3d_branch2a[0][0]']         
 ion)     

 add_8 (Add)                    (None, 14, 14, 1024  0           ['bn4b_branch2c[0][0]',          
                                )                                 'activation_24[0][0]']          
                                                                                                  
 activation_27 (Activation)     (None, 14, 14, 1024  0           ['add_8[0][0]']                  
                                )                                                                 
                                                                                                  
 res4c_branch2a (Conv2D)        (None, 14, 14, 256)  262400      ['activation_27[0][0]']          
                                                                                                  
 bn4c_branch2a (BatchNormalizat  (None, 14, 14, 256)  1024       ['res4c_branch2a[0][0]']         
 ion)                                                                                             
          

                                                                                                  
 res4f_branch2a (Conv2D)        (None, 14, 14, 256)  262400      ['activation_36[0][0]']          
                                                                                                  
 bn4f_branch2a (BatchNormalizat  (None, 14, 14, 256)  1024       ['res4f_branch2a[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_37 (Activation)     (None, 14, 14, 256)  0           ['bn4f_branch2a[0][0]']          
                                                                                                  
 res4f_branch2b (Conv2D)        (None, 14, 14, 256)  590080      ['activation_37[0][0]']          
                                                                                                  
 bn4f_bran

 ion)                                                                                             
                                                                                                  
 activation_46 (Activation)     (None, 7, 7, 512)    0           ['bn5c_branch2a[0][0]']          
                                                                                                  
 res5c_branch2b (Conv2D)        (None, 7, 7, 512)    2359808     ['activation_46[0][0]']          
                                                                                                  
 bn5c_branch2b (BatchNormalizat  (None, 7, 7, 512)   2048        ['res5c_branch2b[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_47 (Activation)     (None, 7, 7, 512)    0           ['bn5c_branch2b[0][0]']          
          

In [18]:
base_model.load_weights("H:/Resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [19]:
for layer in base_model.layers:
    layer.trainable = False

In [20]:
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001D5A51C7FA0> False
<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x000001D5A51C4970> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001D5A51C7E20> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001D5A6216200> False
<keras.layers.core.activation.Activation object at 0x000001D5A6214820> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001D5A6214610> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001D5A61B1330> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001D5A62031F0> False
<keras.layers.core.activation.Activation object at 0x000001D5A6203880> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x000001D5A6280850> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000001D5A6280640> False
<keras.layers.core.activation.Activation object at 0x000

In [21]:
# opt = SGD(learning_rate=0.01, momentum=0.9)
# model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
#from tensorflow.keras.optimizers import SGD

opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [22]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [23]:


# mc = ModelCheckpoint('H:/Dataset/best_model.h5', monitor='val_accuracy', mode='')
mc = ModelCheckpoint('H:Resnet50/model_weight/best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)



In [ ]:
H = model.fit(train_generator,validation_data=val_generator,epochs=50,verbose=1,callbacks=[mc,es])

Epoch 1/50
84/84 [==============================] - 1165s 13s/step - loss: 0.8296 - accuracy: 0.7805 - val_loss: 0.1938 - val_accuracy: 0.9817
Epoch 2/50
84/84 [==============================] - 1089s 13s/step - loss: 0.2062 - accuracy: 0.9719 - val_loss: 0.1829 - val_accuracy: 0.9700
Epoch 3/50
84/84 [==============================] - 1096s 13s/step - loss: 0.1790 - accuracy: 0.9788 - val_loss: 0.1339 - val_accuracy: 0.9950
Epoch 4/50
84/84 [==============================] - 1117s 13s/step - loss: 0.1569 - accuracy: 0.9862 - val_loss: 0.1348 - val_accuracy: 0.9933
Epoch 5/50
84/84 [==============================] - 1046s 12s/step - loss: 0.1506 - accuracy: 0.9893 - val_loss: 0.1373 - val_accuracy: 0.9967
Epoch 6/50
84/84 [==============================] - 1051s 12s/step - loss: 0.1548 - accuracy: 0.9874 - val_loss: 0.1260 - val_accuracy: 0.9983
Epoch 7/50
84/84 [==============================] - 1088s 13s/step - loss: 0.1391 - accuracy: 0.9940 - val_loss: 0.1286 - val_accuracy: 0.9950

In [1]:
model.load_weights("H:/Resnet50/model_weight/best_model.h5")

NameError: name 'model' is not defined

In [32]:
model.evaluate(test_generator)

41/41 [==============================] - 187s 5s/step - loss: 0.1226 - accuracy: 0.9970


[0.12261779606342316, 0.9970059990882874]

In [33]:
model_json = model.to_json()
with open("H:/Resnet50/model.json","w") as json_file:
  json_file.write(model_json)

In [34]:
from keras.models import model_from_json

In [35]:
def predict_(image_path):
    # Load the Model from Json File
    json_file = open('H:/Resnet50/model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    # Load the weights
    model_c.load_weights("H:/Resnet50/best_model.h5")
    # Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    # Load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    cv2_imshow(image)
    # Predict the image
    preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    predicted_class_index = np.argmax(preds)  # Get the index of the predicted class
    # Assuming you have a list of class names
    class_names = ['Advance Direction Sign', 'Bus Stop', 'Crossroad with a minor ahead', 'Direction sign (minor routes)', 'Disabled', 'Give way', 'Hospital', 'Keep Left', 'Keep Right', 'Mosque', 'Narrow bridge ahead', 'No Entry', 'No left turn', 'No Overtaking', 'NO Parking', 'No parkingg', 'No pedestrians', 'No rickshaws', 'No rickshaws and Cycles', 'No right turn', 'NO Stopping', 'No trucks', 'No U turn', 'No use of horn', 'One way street', 'Other danger ahead', 'Parking', 'parkingg', 'Pass either side', 'Pedestrian Crossing', 'Railway level crossing with gate or barrier', 'Reduce Speed', 'Road hump', 'School', 'Sharp bend to the left', 'Sharp bend to the left ahead', 'Sharp bend to the right', 'Sharp bend to the right ahead', 'Sharp change of multiplayer chevron the right', 'Side road on the left ahead', 'Speed limit', 'Traffic merges from Right', 'Turn left', 'Turn Left Ahead', 'Turn Right', 'Turn Right Ahead', 'U-turn']  # Replace with your actual class names
    predicted_class_name = class_names[predicted_class_index]
    print("Predicted Label:", predicted_class_name)


In [30]:
# import matplotlib.pyplot as plt

# def predict_(image_path):
#     # Load the Model from Json File
#     json_file = open('H:/Dataset/model.json', 'r')
#     model_json_c = json_file.read()
#     json_file.close()
#     model_c = model_from_json(model_json_c)
#     # Load the weights
#     model_c.load_weights("H:/Dataset/best_model.h5")
#     # Compile the model
#     opt = SGD(lr=1e-4, momentum=0.9)
#     model_c.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#     # Load the image you want to classify
#     image = cv2.imread(image_path)
#     image = cv2.resize(image, (224, 224))
#     plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#     plt.axis('off')
#     plt.show()
#     # Predict the image
#     preds = model_c.predict(np.expand_dims(image, axis=0))[0]
#     predicted_class_index = np.argmax(preds)  # Get the index of the predicted class
#     # Assuming you have a list of class names
#     class_names = ['Advance Direction Sign', 'Bus Stop', 'Crossroad with a minor ahead', 'Direction sign (minor routes)', 'Disabled', 'Give way', 'Hospital', 'Keep Left', 'Keep Right', 'Mosque', 'Narrow bridge ahead', 'No Entry', 'No left turn', 'No Overtaking', 'NO Parking', 'No parkingg', 'No pedestrians', 'No rickshaws', 'No rickshaws and Cycles', 'No right turn', 'NO Stopping', 'No trucks', 'No U turn', 'No use of horn', 'One way street', 'Other danger ahead', 'Parking', 'parkingg', 'Pass either side', 'Pedestrian Crossing', 'Railway level crossing with gate or barrier', 'Reduce Speed', 'Road hump', 'School', 'Sharp bend to the left', 'Sharp bend to the left ahead', 'Sharp bend to the right', 'Sharp bend to the right ahead', 'Sharp change of multiplayer chevron the right', 'Side road on the left ahead', 'Speed limit', 'Traffic merges from Right', 'Turn left', 'Turn Left Ahead', 'Turn Right', 'Turn Right Ahead', 'U-turn']  # Replace with your actual class names
#     predicted_class_name = class_names[predicted_class_index]
#     print("Predicted Label:", predicted_class_name)


In [36]:
import matplotlib.pyplot as plt
import tensorflow as tf

def predict_(image_path):
    # Load the Model from Json File
    json_file = open('H:/Resnet50/model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    # Load the weights
    model_c.load_weights("H:/Resnet50/best_model.h5")
    # Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    # Load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    # Predict the image
    with tf.device('/CPU:0'):  # Execute on CPU to avoid retracing warning
        preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    predicted_class_index = np.argmax(preds)  # Get the index of the predicted class
    # Assuming you have a list of class names
    class_names = ['Bus Stop', 'Crossroad with a minor ahead', 'Give way', 
                   'Hospital', 'Mosque', 'Narrow bridge ahead', 'No left turn', 
                   'No Overtaking', 'NO Parking', 'No rickshaws and Cycles', 'No right turn',
                   'NO Stopping', 'No trucks', 'No U turn', 'Other danger ahead', 'Parking', 
                   'Pedestrian Crossing', 'Railway level crossing with gate or barrier',
                   'Road hump', 'School', 'Sharp bend to the left ahead', 
                   'Sharp bend to the right ahead', 'Turn left', 'Turn Right', 'U-turn']  # Replace with your actual class names
    predicted_class_name = class_names[predicted_class_index]
    print("Predicted Label:", predicted_class_name)


In [37]:
predict_("H:/Resnet50/test/Crossroad with a minor ahead/Crossroad with a minor ahead_0_1241.png")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'H:/Resnet50/best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [38]:
predict_("H:/Resnet50/test/Crossroad with a minor ahead/Crossroad with a minor ahead_0_1241.png")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'H:/Resnet50/best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)